In [1]:
import pandas as pd
import numpy as np

moviesOriginal = pd.read_csv("../movies.csv/movies.csv")

moviesOriginal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773743 entries, 0 to 773742
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    773743 non-null  int64  
 1   title                 773739 non-null  object 
 2   genres                544554 non-null  object 
 3   original_language     773743 non-null  object 
 4   overview              647617 non-null  object 
 5   popularity            773743 non-null  float64
 6   production_companies  360920 non-null  object 
 7   release_date          711558 non-null  object 
 8   budget                773743 non-null  float64
 9   revenue               773743 non-null  float64
 10  runtime               732454 non-null  float64
 11  status                773743 non-null  object 
 12  tagline               115157 non-null  object 
 13  vote_average          773743 non-null  float64
 14  vote_count            773743 non-null  float64
 15  

In [2]:
movies = moviesOriginal[['id', 'title', 'genres', 'original_language', 'production_companies', 'release_date', 'budget', 'revenue', 'runtime', 'vote_average', 'vote_count']]

movies.info()
movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773743 entries, 0 to 773742
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    773743 non-null  int64  
 1   title                 773739 non-null  object 
 2   genres                544554 non-null  object 
 3   original_language     773743 non-null  object 
 4   production_companies  360920 non-null  object 
 5   release_date          711558 non-null  object 
 6   budget                773743 non-null  float64
 7   revenue               773743 non-null  float64
 8   runtime               732454 non-null  float64
 9   vote_average          773743 non-null  float64
 10  vote_count            773743 non-null  float64
dtypes: float64(5), int64(1), object(5)
memory usage: 64.9+ MB


,id,title,genres,original_language,production_companies,release_date,budget,revenue,runtime,vote_average,vote_count
0,338953,Fantastic Beasts: The Secrets of Dumbledore,Fantasy-Adventure-Action,en,Warner Bros. Pictures-Heyday Films,2022-04-06,200000000.0,380000000.0,142.0,6.8,1410.0
1,526896,Morbius,Action-Science Fiction-Fantasy,en,Columbia Pictures-Avi Arad Productions-Matt To...,2022-03-30,75000000.0,161000000.0,105.0,6.4,1424.0
2,675353,Sonic the Hedgehog 2,Action-Adventure-Family-Comedy,en,Original Film-Blur Studio-Marza Animation Plan...,2022-03-30,110000000.0,375000000.0,122.0,7.7,1849.0
3,752623,The Lost City,Action-Adventure-Comedy-Romance,en,Paramount-Fortis Films-3dot Productions-Exhibi...,2022-03-24,74000000.0,164289828.0,112.0,6.8,959.0
4,818397,Memory,Action-Thriller-Crime,en,Black Bear Pictures-Welle Entertainment-Savill...,2022-04-28,0.0,8597928.0,114.0,7.3,200.0


Перевіримо, чи має дата фрейм пропущені значення:

In [3]:
movies.isnull().values.any()

True

Тепер перевіримо наявність від'ємних значень:

In [4]:
(movies.select_dtypes(include=np.number) < 0).values.any()

True

In [5]:
movies[movies['budget'] < 0].head()

,id,title,genres,original_language,production_companies,release_date,budget,revenue,runtime,vote_average,vote_count


In [6]:
movies[['budget', 'runtime']].corr()

,budget,runtime
budget,1.000000,-0.000048
runtime,-0.000048,1.000000


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
movies = movies[movies['revenue'].notna()]

In [9]:
numeric_columns = movies.select_dtypes(include=['number']).columns

means = movies[numeric_columns].mean()

movies[numeric_columns] = movies[numeric_columns].fillna(means)

In [28]:

X = movies[['budget', 'runtime', 'vote_average', 'genres']]
Y = movies[['revenue']]



In [30]:
res = X.genres.str.get_dummies(sep='-').add_prefix('genres_')

X = pd.concat([X, res], axis = 1)
X.drop(columns=['genres'], inplace=True)
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 773743 entries, 0 to 773742
Data columns (total 41 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   budget                  773743 non-null  float64
 1   runtime                 773743 non-null  float64
 2   vote_average            773743 non-null  float64
 3   genres_Action           773743 non-null  int64  
 4   genres_Adventure        773743 non-null  int64  
 5   genres_Animation        773743 non-null  int64  
 6   genres_Comedy           773743 non-null  int64  
 7   genres_Crime            773743 non-null  int64  
 8   genres_Documentary      773743 non-null  int64  
 9   genres_Drama            773743 non-null  int64  
 10  genres_Family           773743 non-null  int64  
 11  genres_Fantasy          773743 non-null  int64  
 12  genres_History          773743 non-null  int64  
 13  genres_Horror           773743 non-null  int64  
 14  genres_Music        

,budget,runtime,vote_average,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,...,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western
0,200000000.0,142.0,6.8,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,75000000.0,105.0,6.4,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,110000000.0,122.0,7.7,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,74000000.0,112.0,6.8,1,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,114.0,7.3,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 5)

In [32]:
LinReg = LinearRegression().fit(X_train, Y_train)

In [33]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

PolReg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
PolReg.fit(X_test, Y_test)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [34]:
predictions = [LinReg.predict(X_test),
               PolReg.predict(X_test)]

In [35]:
from sklearn.metrics import r2_score
from numpy import sqrt

RSE_scores = []
R2_scores = []

for prediction in predictions:
    y_true = np.array(Y_test)
    y_predicted = np.array(prediction)
    RSE_scores.append(sqrt((np.sum(np.square(y_true - y_predicted)))/(len(y_true) - 2)))
    R2_scores.append(r2_score(Y_test, prediction))

print('RSE: ', RSE_scores)
print('R2: ', R2_scores)
print('RSE index min: ', np.argmin(RSE_scores) +1, RSE_scores[np.argmin(RSE_scores)])
print('R2 index max: ', np.argmax(R2_scores) + 1, R2_scores[np.argmax(R2_scores)])

RSE:  [16363518.116209924, 12201600.883281613]
R2:  [0.313870729750268, 0.6185073162143048]
RSE index min:  2 12201600.883281613
R2 index max:  2 0.6185073162143048
